## Encontrar alpha y beta para cada fecha 

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#  Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

#  Cargar los datos del archivo CSV
Fecha = '2023-01-30'
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')
row_day_solcast = df_solcast.loc[df_solcast['Date'] == Fecha]
row_day_fronius = df_SFV.loc[df_SFV['Date'] == Fecha]
# reiniciar el index
row_day_solcast = row_day_solcast.reset_index(drop=True)
row_day_fronius = row_day_fronius.reset_index(drop=True)
# Odenernar row_day_solcast de menor a mayor hora
row_day_solcast = row_day_solcast.sort_values(by='Time')
row_day_solcast = row_day_solcast.reset_index(drop=True)
# Crear un nuevo dataframe con los datos de solcast y fronius a partir del indice 
df = pd.DataFrame()
df['Time'] = row_day_solcast['Time']
df['GHI'] = row_day_solcast['GHI']
df['air_temp'] = row_day_solcast['air_temp']
df['cloud_opacity'] = row_day_solcast['cloud_opacity']
df['Producción_fotovoltaica_SFV'] = row_day_fronius['Producción_fotovoltaica_SFV']

/var/folders/pz/w3bcsy5x2wgb_7c0fs6qmkp80000gn/T/ipykernel_83067/48519849.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 🌕

In [14]:
import pandas as pd

# Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')

# Obtener todas las fechas únicas en df_SFV
fechas_unicas = df_SFV['Date'].unique()

# Crear un DataFrame vacío para almacenar los resultados
resultados = pd.DataFrame()

# Iterar sobre cada fecha única
for Fecha in fechas_unicas:
    print(Fecha)
    # Filtrar los datos para la fecha actual
    row_day_solcast = df_solcast.loc[df_solcast['Date'] == Fecha].copy()
    row_day_fronius = df_SFV.loc[df_SFV['Date'] == Fecha].copy()

    # Reiniciar el index
    row_day_solcast.reset_index(drop=True, inplace=True)
    row_day_fronius.reset_index(drop=True, inplace=True)

    # Ordenar row_day_solcast de menor a mayor hora
    row_day_solcast.sort_values(by='Time', inplace=True)
    row_day_solcast.reset_index(drop=True, inplace=True)

    # Crear un nuevo DataFrame con los datos de solcast y fronius
    df_temporal = pd.DataFrame()
    df_temporal['Time'] = row_day_solcast['Time']
    df_temporal['GHI'] = row_day_solcast['GHI']
    df_temporal['air_temp'] = row_day_solcast['air_temp']
    df_temporal['cloud_opacity'] = row_day_solcast['cloud_opacity']
    df_temporal['Producción_fotovoltaica_SFV'] = row_day_fronius['Producción_fotovoltaica_SFV']
    #clearsky_ghi
    df_temporal['clearsky_ghi'] = row_day_solcast['clearsky_ghi']

    # Calculo de energía
    # Parametros SFV
    efficiency = 0.2094  # Eficiencia del panel solar
    catidad_pv = 30
    area = 2.274 * 1.134

    # Normalizar 'cloud_opacity' de 0 a 1  en donde 1 es el valor más alto
    df_temporal['cloud_opacity'] = df_temporal['cloud_opacity'] / df_temporal['cloud_opacity'].max()
    # Normalizar 'air_temp' de 0 a 1  en donde 1 es el valor más alto
    df_temporal['air_temp'] = df_temporal['air_temp'] / df_temporal['air_temp'].max()

    # Energía calculada a partir de Solcast
    alpha = 0
    beta = 0
    df_temporal['P_cloud_opacity_temp'] = df_temporal['GHI'] * efficiency * area * catidad_pv * (
                1 - beta * df_temporal['cloud_opacity']) * (1 - alpha * df_temporal['air_temp'])

    # Energía calculada a partir de Fronius
    df_temporal['Potencia'] = df_temporal['Producción_fotovoltaica_SFV'] / (5 / 60)

    # Energía calculada a partir de Fronius
    energia_fronius = df_temporal['Producción_fotovoltaica_SFV'].sum()
    # Horas de brillo solar
    hbs = energia_fronius / (30 * 0.54)
    print("Horas de brillo solar =", hbs / 1000)
    print("Producción fotovoltaica SVF [kWh/día] =", energia_fronius / 1000)
    # Suma de la potencia generada solcast
    energia = df_temporal['P_cloud_opacity_temp'] * (5 / 60)
    print("Energía calculada solacast [kWh/día] =", energia.sum() / 1000)
    # Error porcentual
    error = abs(energia.sum() - energia_fronius) / energia_fronius * 100
    print('Error porcentual:', error, '%')

    # Alpha y beta óptimos
    from scipy.optimize import minimize

    # Función para minimizar la diferencia entre las energías calculadas
    def objective_function(params, df):
        alpha, beta = params
        energia_solcast = df['GHI'] * efficiency * area * catidad_pv * (1 - beta * df['cloud_opacity']) * (
                    1 - alpha * df['air_temp']) * (5 / 60)
        diff = energia_solcast.sum() - energia_fronius
        return abs(diff)

    # Valores iniciales de alpha y beta
    initial_guess = [0.1, 0.1]

    # Minimización de la función objetivo
    result = minimize(objective_function, initial_guess, args=(df_temporal,),
                      bounds=((0, 1), (-1, 0)))
    alpha_optimal, beta_optimal = result.x

    print("Alpha óptimo:", alpha_optimal)
    print("Beta óptimo:", beta_optimal)
    print("***************************")

    # Agregar el DataFrame temporal al DataFrame de resultados
    resultados = pd.concat([resultados, df_temporal])
    # Agregar energia_fronius, Fecha, hbs, alpha_optimal, beta_optimal y error 
    resultados['Energia_fronius'] = energia_fronius
    resultados['Fecha'] = Fecha
    resultados['Horas_brillo_solar'] = hbs/1000
    resultados['Alpha_optimo'] = alpha_optimal
    resultados['Beta_optimo'] = beta_optimal
    resultados['Error'] = error
    

# Restablecer el índice de los resultados
resultados.reset_index(drop=True, inplace=True)

# Mostrar los resultados
print(resultados)

# Guardar los resultados en un archivo CSV
resultados.to_csv('../Data/processed/cofactores/resultados.csv', index=False)


2023-01-01
Horas de brillo solar = 3.7708709876543205
Producción fotovoltaica SVF [kWh/día] = 61.08811
Energía calculada solacast [kWh/día] = 73.79544455829001
Error porcentual: 20.801649548971163 %
Alpha óptimo: 0.19770557217384027
Beta óptimo: -1.0234405153332126e-09
***************************
2023-01-02
Horas de brillo solar = 1.723062345679012
Producción fotovoltaica SVF [kWh/día] = 27.913610000000002
Energía calculada solacast [kWh/día] = 26.43217423308
Error porcentual: 5.307216683617775 %
Alpha óptimo: 0.08182920840495778
Beta óptimo: -0.18170791595042246
***************************
2023-01-03
Horas de brillo solar = 3.6050512345679007
Producción fotovoltaica SVF [kWh/día] = 58.401830000000004
Energía calculada solacast [kWh/día] = 62.616443801184
Error porcentual: 7.216578318152006 %
Alpha óptimo: 0.09582928263325867
Beta óptimo: -0.041707173667515626
***************************
2023-01-04
Horas de brillo solar = 4.3621074074074055
Producción fotovoltaica SVF [kWh/día] = 70.66